In [1]:
seriesId = 'NHSDPCS' #FRED data series ID
apiKey = "c71c855489afb5327e992f3829276b54" #personal key, need to get one per app

In [2]:
#get FRED series data and return data frame
import certifi
import urllib3
import json

def getFREDSeriesData (seriesId, apiKey): 
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
    requestUrl = "https://api.stlouisfed.org/fred/series/observations?series_id=" + seriesId + "&api_key=" + apiKey + "&file_type=json"

    payload = http.request('GET', requestUrl)
    rawData = json.loads(payload.data.decode('utf-8'))
    df = pd.json_normalize(rawData['observations'])

    return df

In [3]:
#clean up FRED data
import pandas as pd

def cleanFREDSeriesData (df):
    df.index.name = "observationId"
    df.drop(['realtime_start', 'realtime_end'],axis=1, inplace=True)
    df['value'] = pd.to_numeric(df['value']) * 1000 #convert data from thousands
    #df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    df.sort_values('date', inplace=True)

    return df

In [4]:
#date filter df
import pandas as pd
import datetime

def dateFilterDF(df, dateColumn, startDate, endDate):
    mask = (df[dateColumn] >= startDate) & (df[dateColumn] <= endDate)
    df = df.loc[mask]
    return df

In [5]:
#detrend specified column of df, return df with "detrendValue" column
from scipy.signal import detrend

def detrendColumn(df, column):
    df['detrendValue'] = detrend(df[column])
    return df

In [6]:
newHmSalesMaster = getFREDSeriesData(seriesId, apiKey) #get FRED new home sales
newHmSalesMaster = cleanFREDSeriesData(newHmSalesMaster) #cleanup FRED new home sales

In [7]:
import pandas as pd

validationDataSet = dateFilterDF(newHmSalesMaster, 'date', '1999-01-01', '2010-12-01')
validationDataSet = detrendColumn(validationDataSet, 'value')

movingAvg = validationDataSet['detrendValue'].rolling(3).mean()

In [8]:
from statsmodels.tsa.arima_model import ARIMA
from matplotlib import pyplot

trainingDataSet = dateFilterDF(newHmSalesMaster, 'date', '2011-01-01', '2015-12-01')
trainingDataSet = detrendColumn(trainingDataSet, 'value')

# fit model
model = ARIMA(trainingDataSet, order=(5,1,0))
model_fit = model.fit(disp=0)
print(model_fit.summary())
# plot residual errors
residuals = DataFrame(model_fit.resid)
residuals.plot()
pyplot.show()
residuals.plot(kind='kde')
pyplot.show()
print(residuals.describe())

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
import matplotlib.pyplot as plt

plt.plot(newHmSales['detrendValue'])
plt.plot(movingAvg, color='red')